In [8]:
import mujoco
import glfw

def key_callback(window, key, scancode, action, mods):
    if key == glfw.KEY_ESCAPE and action == glfw.PRESS:
        glfw.set_window_should_close(window, True)
    elif key == glfw.KEY_UP and action == glfw.PRESS:
        print("Up key pressed")

def mouse_callback(window, xpos, ypos):
    print("Mouse position:", xpos, ypos)


xml = """
<mujoco>
  <option timestep=".001">
    <flag energy="enable" contact="disable"/>
  </option>

  <default>
    <joint type="hinge" axis="0 -1 0"/>
    <geom type="capsule" size=".02"/>
  </default>

  <worldbody>
    <light pos="0 -.4 1"/>
    <camera name="fixed" pos="0 -1 0" xyaxes="1 0 0 0 0 1"/>
    <body name="0" pos="0 0 .2">
      <joint name="root"/>
      <geom fromto="-.2 0 0 .2 0 0" rgba="1 1 0 1"/>
      <geom fromto="0 0 0 0 0 -.25" rgba="1 1 0 1"/>
      <body name="1" pos="-.2 0 0">
        <joint/>
        <geom fromto="0 0 0 0 0 -.2" rgba="1 0 0 1"/>
      </body>
      <body name="2" pos=".2 0 0">
        <joint/>
        <geom fromto="0 0 0 0 0 -.2" rgba="0 1 0 1"/>
      </body>
      <body name="3" pos="0 0 -.25">
        <joint/>
        <geom fromto="0 0 0 0 0 -.2" rgba="0 0 1 1"/>
      </body>
    </body>
  </worldbody>
</mujoco>
"""

model = mujoco.MjModel.from_xml_string(xml)
data = mujoco.MjData(model)

# GLFW 윈도우 생성 및 초기화
if not glfw.init():
    print("Could not initialize GLFW")
    exit()

window = glfw.create_window(1200, 900, "MuJoCo Window", None, None)
if not window:
    glfw.terminate()
    print("Could not create GLFW window")
    exit()

glfw.make_context_current(window)
glfw.set_key_callback(window, key_callback)
glfw.set_cursor_pos_callback(window, mouse_callback)


mujoco.mj_resetData(model, data)
del scene
del context
scene = mujoco.MjvScene(model, maxgeom=10000)
camera = mujoco.MjvCamera()  # 카메라 설정
viewport = mujoco.MjrRect(0, 0, 1200, 900)
context = mujoco.MjrContext(model, mujoco.mjtFontScale.mjFONTSCALE_100)

data.joint('root').qvel = 10

while not glfw.window_should_close(window):
    # print("Time: ", data.time)
    # 시뮬레이션 스텝 수행
    mujoco.mj_step(model, data)

    # 렌더링
    mujoco.mjv_updateScene(model, data, mujoco.MjvOption(), None, camera, mujoco.mjtCatBit.mjCAT_ALL, scene)
    mujoco.mjr_render(viewport, scene, context)

    glfw.swap_buffers(window)
    glfw.poll_events()

glfw.terminate()
